In [16]:
%%capture
!pip install -U -q sagemaker
!pip install -U -q transformer
!pip install torch==1.11

In [18]:
import sys
import transformers
import sagemaker
import boto3
import torch

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sess_bucket = sagemaker_session.default_bucket()
print(torch.__version__)

1.11.0+cu102


In [19]:
tokenizer = transformers.AutoTokenizer.from_pretrained("BAAI/bge-m3")

model = transformers.AutoModel.from_pretrained(
    "BAAI/bge-m3", return_dict=False
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
inputs = "Hello, SageMaker"
encoded_input = tokenizer(inputs, return_tensors='pt')
with torch.no_grad():
    outputs = model(**encoded_input)
print(str(outputs[1].shape))

torch.Size([1, 1024])


# Test Endpoint

In [1]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

seq_0 = "Welcome to AWS Summit San Francisco 2022! Thank you for attending the workshop on using Huggingface transformers on Inferentia instances."
seq_1 = seq_0
payload = (seq_0, seq_1)

serializer = JSONSerializer()
serialized_input_data = serializer.serialize(payload)
print(serialized_input_data)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
["Welcome to AWS Summit San Francisco 2022! Thank you for attending the workshop on using Huggingface transformers on Inferentia instances.", "Welcome to AWS Summit San Francisco 2022! Thank you for attending the workshop on using Huggingface transformers on Inferentia instances."]


In [12]:
import json
import boto3
from io import BytesIO

# Set up the SageMaker runtime client
runtime = boto3.client('runtime.sagemaker')


In [14]:

# Define the endpoint name
endpoint_name = 'bge-m3-inf1-202405-2805-4315'

# Make a prediction request to the endpoint
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=serialized_input_data
)

# Deserialize the response
deserializer = JSONDeserializer()
response_body = BytesIO(response['Body'].read())
result = deserializer.deserialize(response_body, 'application/json')

# Print the result
print(result)

['"(tensor([[[-0.2425,  0.1145, -0.7044,  ...,  0.9198, -0.1090,  1.0912],\\n         [ 0.0309, -0.6001, -0.5553,  ...,  1.6043, -0.5135,  1.2376],\\n         [-0.0437, -0.6375, -0.4062,  ...,  1.2313, -0.6825,  0.7152],\\n         ...,\\n         [ 0.1653,  0.6219, -0.5957,  ...,  1.2212,  0.0403,  0.9938],\\n         [ 0.1653,  0.6219, -0.5957,  ...,  1.2212,  0.0403,  0.9938],\\n         [ 0.1653,  0.6219, -0.5957,  ...,  1.2212,  0.0403,  0.9938]]]), tensor([[0.1304, 0.0103, 0.0873,  ..., 0.4091, 0.5241, 0.0126]]))"', 'application/json']


In [15]:
# Define the endpoint name
endpoint_name = 'bge-m3-inf1-202405-2611-5529'

# Make a prediction request to the endpoint
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=serialized_input_data
)

# Deserialize the response
deserializer = JSONDeserializer()
response_body = BytesIO(response['Body'].read())
result = deserializer.deserialize(response_body, 'application/json')

# Print the result
print(result)

(tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]]), tensor([[nan, nan, nan,  ..., nan, nan, nan]]))
